In [4]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Cuarto paso: correr el modelo

In [5]:
l_rates = [0.001, 0.01]
epochs = [10,20,30] #10,20,30
batch_sizes = [8,16,30]#[8,16,30]

experiment = 1 #Número donde inician los experimentos
for batch in batch_sizes:
    for epoch in epochs:
        for learning_rate in l_rates:
        
            print("*"*20)
            print("EXPERIMENTO: Batch:", batch, "Epoc: ", epoch, "learning_rate: ", learning_rate)
            
            #Cargar los parametros cada que inicia un experimento
            params = utils.yaml_to_dict(os.path.join('..','config.yml'))
            params['data_dir'] = os.path.join('..',params['data_dir'])
            
            experiment_name = 'Experimento10-Configuracion'+str(experiment)
            params['model_dir'] = os.path.join('..','experiments-weights',experiment_name)
            params['batch_size'] = batch
            params['num_epochs'] = epoch
            params['learning_rate'] = learning_rate
            
            #Limpiar Sesión
            tf.keras.backend.clear_session()
            
            #Cargar el modelo
            inputs = tf.keras.layers.Input(shape=(110,480,3))
            net = model.ModelArchitecture(num_classes=params['num_classes'])
            x = net(inputs, training=False)
            
            #Generar conjuntos de entrenamiento y validación y llevarlos a un generador
            preprocessing.split_data(params)
            train_generator = data.make_datagenerator(params, mode='training')
            val_generator = data.make_datagenerator(params,mode='validation')
            
            #Crear el directorio que guarda los checkpoints
            if not os.path.exists(params['model_dir']):
                os.makedirs(params['model_dir'])

            #Definir los callbacks
            cp_callback = tf.keras.callbacks.ModelCheckpoint(
                os.path.join(params['model_dir'], 'tf_ckpt'), 
                save_weights_only=True, 
                verbose=1,
                period=5)

            tb_callback = tf.keras.callbacks.TensorBoard(
                os.path.join(params['model_dir'], 'logs'))

            #Definir la función de optimización
            optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

            #Definir los pasos para entrenamiento y validación
            steps_per_epoch = train_generator.n // params['batch_size']
            validation_steps = val_generator.n // params['batch_size']

            #Entrenar el modelo
            net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
            net.fit_generator(
                train_generator, 
                steps_per_epoch=steps_per_epoch, 
                epochs=params['num_epochs'],
                workers=4,
                validation_data=val_generator, 
                validation_steps=validation_steps,
                callbacks=[cp_callback,tb_callback])
            
    
            #Obtener las métricas de resultados en el conjunto de validación
            params['batch_size'] = val_generator.n
            params['shuffle'] = False
            val_generator = data.make_datagenerator(params,mode='validation')
            Y_pred = net.predict_generator(val_generator, steps=1)
            y_pred = np.argmax(Y_pred, axis=1)
            
            print(confusion_matrix(val_generator.data, y_pred))
            target_names = ["arranca", "acelera", "pita", "izquierda","frena", "detente", "retrocede", "gira", "avanza", "derecha"]
            print(classification_report(val_generator.data, y_pred, target_names=target_names))
            
            #Aumentar el contador de experimentos
            experiment += 1

********************
EXPERIMENTO: Batch: 8 Epoc:  10 learning_rate:  0.001
Found 2960 images.
Found 30 images.
Epoch 1/10
370/370 [==============================] - 21s 57ms/step - loss: 0.5944 - sparse_categorical_accuracy: 0.8159 - val_loss: 0.3312 - val_sparse_categorical_accuracy: 0.8750
Epoch 2/10
370/370 [==============================] - 21s 58ms/step - loss: 0.1370 - sparse_categorical_accuracy: 0.9571 - val_loss: 0.0801 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/10
370/370 [==============================] - 21s 57ms/step - loss: 0.0717 - sparse_categorical_accuracy: 0.9764 - val_loss: 0.0873 - val_sparse_categorical_accuracy: 0.9583
Epoch 4/10
370/370 [==============================] - 21s 56ms/step - loss: 0.0464 - sparse_categorical_accuracy: 0.9841 - val_loss: 0.0205 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/10
368/370 [============================>.] - ETA: 0s - loss: 0.0340 - sparse_categorical_accuracy: 0.9888- ETA: 4s - loss: 0.03
Epoch 00005: saving mode

370/370 [==============================] - 22s 60ms/step - loss: 0.0819 - sparse_categorical_accuracy: 0.9818 - val_loss: 0.6080 - val_sparse_categorical_accuracy: 0.9167
Found 30 images.
[[2 0 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0 0]
 [0 0 0 1 2 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 1 0 2 0]
 [0 0 0 0 0 0 0 0 0 2]]
              precision    recall  f1-score   support

     arranca       1.00      1.00      1.00         2
     acelera       1.00      1.00      1.00         4
        pita       1.00      1.00      1.00         4
   izquierda       0.80      1.00      0.89         4
       frena       1.00      0.67      0.80         3
     detente       0.75      1.00      0.86         3
   retrocede       0.75      1.00      0.86         3
        gira       0.00      0.00      0.00         2
      avanza       1.00      0.67      0.80         3
     derecha       0.67      1.00      0

C:\Users\PC\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Found 2960 images.
Found 30 images.
Epoch 1/20
370/370 [==============================] - 22s 59ms/step - loss: 0.5948 - sparse_categorical_accuracy: 0.8166 - val_loss: 0.4386 - val_sparse_categorical_accuracy: 0.9167
Epoch 2/20
370/370 [==============================] - 23s 63ms/step - loss: 0.1298 - sparse_categorical_accuracy: 0.9598 - val_loss: 0.4110 - val_sparse_categorical_accuracy: 0.9167
Epoch 3/20
370/370 [==============================] - 19s 50ms/step - loss: 0.0746 - sparse_categorical_accuracy: 0.9750 - val_loss: 0.2231 - val_sparse_categorical_accuracy: 0.9583
Epoch 4/20
370/370 [==============================] - 20s 53ms/step - loss: 0.0514 - sparse_categorical_accuracy: 0.9855 - val_loss: 0.2767 - val_sparse_categorical_accuracy: 0.9583
Epoch 5/20
368/370 [============================>.] - ETA: 0s - loss: 0.0280 - sparse_categorical_accuracy: 0.9929
Epoch 00005: saving model to ..\experiments-weights\Experimento10-Configuracion3\tf_ckpt

Consider using a TensorFlow opt

370/370 [==============================] - 20s 55ms/step - loss: 0.1291 - sparse_categorical_accuracy: 0.9605 - val_loss: 0.2368 - val_sparse_categorical_accuracy: 0.8750
Epoch 3/20
370/370 [==============================] - 19s 52ms/step - loss: 0.0833 - sparse_categorical_accuracy: 0.9753 - val_loss: 0.1829 - val_sparse_categorical_accuracy: 0.9583
Epoch 4/20
370/370 [==============================] - 18s 48ms/step - loss: 0.0544 - sparse_categorical_accuracy: 0.9838 - val_loss: 0.0273 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/20
369/370 [============================>.] - ETA: 0s - loss: 0.0641 - sparse_categorical_accuracy: 0.9817- ETA: 1s - loss: 0.0526 - sparse_categ
Epoch 00005: saving model to ..\experiments-weights\Experimento10-Configuracion4\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
370/370 [==============================] - 18s 49ms/step - loss: 0.0640 - sparse_categorical_accuracy: 0.9818 - val_loss: 0.2834 - val_sparse_categorical_accuracy: 0.

...

98/98 [==============================] - 10s 98ms/step - loss: 0.0013 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0349 - val_sparse_categorical_accuracy: 0.9667
Epoch 25/30
97/98 [============================>.] - ETA: 0s - loss: 0.0013 - sparse_categorical_accuracy: 1.0000
Epoch 00025: saving model to ..\experiments-weights\Experimento10-Configuracion17\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
98/98 [==============================] - 8s 82ms/step - loss: 0.0013 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0201 - val_sparse_categorical_accuracy: 1.0000
Epoch 26/30
98/98 [==============================] - 9s 87ms/step - loss: 0.0011 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0327 - val_sparse_categorical_accuracy: 0.9667
Epoch 27/30
98/98 [==============================] - 10s 102ms/step - loss: 0.0010 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0243 - val_sparse_categorical_accuracy: 1.0000
Epoch 28/30
98/98 [======================

98/98 [==============================] - 9s 95ms/step - loss: 0.0648 - sparse_categorical_accuracy: 0.9816 - val_loss: 0.0137 - val_sparse_categorical_accuracy: 1.0000gorical_a
Epoch 5/30
97/98 [============================>.] - ETA: 0s - loss: 0.0360 - sparse_categorical_accuracy: 0.9911
Epoch 00005: saving model to ..\experiments-weights\Experimento10-Configuracion18\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
98/98 [==============================] - 13s 129ms/step - loss: 0.0356 - sparse_categorical_accuracy: 0.9912 - val_loss: 0.3237 - val_sparse_categorical_accuracy: 0.9000
Epoch 6/30
98/98 [==============================] - 9s 94ms/step - loss: 0.0388 - sparse_categorical_accuracy: 0.9895 - val_loss: 0.0686 - val_sparse_categorical_accuracy: 0.9333
Epoch 7/30
98/98 [==============================] - 10s 106ms/step - loss: 0.0636 - sparse_categorical_accuracy: 0.9827 - val_loss: 0.1447 - val_sparse_categorical_accuracy: 0.9667
Epoch 8/30
98/98 [================

98/98 [==============================] - 11s 114ms/step - loss: 0.0512 - sparse_categorical_accuracy: 0.9888 - val_loss: 0.1371 - val_sparse_categorical_accuracy: 0.9667
Epoch 14/30
98/98 [==============================] - 9s 97ms/step - loss: 0.0195 - sparse_categorical_accuracy: 0.9963 - val_loss: 1.0768e-05 - val_sparse_categorical_accuracy: 1.0000
Epoch 15/30
97/98 [============================>.] - ETA: 0s - loss: 0.0094 - sparse_categorical_accuracy: 0.9983- ETA: 3s - loss: 0.0
Epoch 00015: saving model to ..\experiments-weights\Experimento10-Configuracion18\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
98/98 [==============================] - 10s 100ms/step - loss: 0.0093 - sparse_categorical_accuracy: 0.9983 - val_loss: 3.4774e-05 - val_sparse_categorical_accuracy: 1.0000
Epoch 16/30
98/98 [==============================] - 11s 111ms/step - loss: 0.0059 - sparse_categorical_accuracy: 0.9997 - val_loss: 7.0370e-06 - val_sparse_categorical_accuracy: 1.0000
Epoch

98/98 [==============================] - 12s 118ms/step - loss: 0.0056 - sparse_categorical_accuracy: 0.9997 - val_loss: 3.2544e-06 - val_sparse_categorical_accuracy: 1.0000
Epoch 22/30
98/98 [==============================] - 14s 146ms/step - loss: 0.0056 - sparse_categorical_accuracy: 0.9997 - val_loss: 3.2742e-06 - val_sparse_categorical_accuracy: 1.0000
Epoch 23/30
98/98 [==============================] - 8s 86ms/step - loss: 5.9268e-05 - sparse_categorical_accuracy: 1.0000 - val_loss: 2.7855e-06 - val_sparse_categorical_accuracy: 1.0000
Epoch 24/30
98/98 [==============================] - 11s 117ms/step - loss: 0.0110 - sparse_categorical_accuracy: 0.9993 - val_loss: 2.7696e-06 - val_sparse_categorical_accuracy: 1.0000
Epoch 25/30
97/98 [============================>.] - ETA: 0s - loss: 0.0056 - sparse_categorical_accuracy: 0.9997
Epoch 00025: saving model to ..\experiments-weights\Experimento10-Configuracion18\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
98/98 

97/98 [============================>.] - ETA: 0s - loss: 0.0111 - sparse_categorical_accuracy: 0.9993
Epoch 00030: saving model to ..\experiments-weights\Experimento10-Configuracion18\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
98/98 [==============================] - 11s 112ms/step - loss: 0.0110 - sparse_categorical_accuracy: 0.9993 - val_loss: 1.9073e-06 - val_sparse_categorical_accuracy: 1.0000
Found 30 images.
[[2 0 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 2]]
              precision    recall  f1-score   support

     arranca       1.00      1.00      1.00         2
     acelera       1.00      1.00      1.00         4
        pita       1.00      1.00      1.00         4
   izquierda       1.00      1.00      1.00         4
       frena       1.00      1.00      1.00         3
     